<a href="https://colab.research.google.com/github/Epilef86/DNC/blob/main/SparkML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  A vantagem de usar spark ML pra fazer execução de algoritmo de machine learn é que podemos distribuir e processar esses dados de forma paralela.O spark ML faz os algotimos e suas previsões, cluesterizações de forma paralela. Outra vantagem é que ele permite que trabalhe não só com python, podemos executar esses algotimos usando R, java, diversas liguagens de programação que o spark nos provém.  

Vamos tentar ver se há alguma fraude na base de transações

1ª coisa é preparar o ambiente

In [20]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


2º Passo: para acessar o sparkui através do colba, é necessário instalar o ngrok, que é uma ferramenta que possibilita que acesse através de um servidor que está no google colab.

In [21]:
!wget -qnc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -n -q ngrok-stable-linux-amd64.zip 

Quando der um refresh os arquivos vão aparecer ao lado, ou seja, o ngrok já está na instalado na máquina

In [22]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .config('spark.ui.port', '4050')
    .appName("SparkUI Introdução")
    .getOrCreate()
)

A única configuração que vai ser colocado a mais é que a porta do sparkui estará na 4050, enquanto padrão (default) é 4040. Faz essa mudança por orientação ao ngrok pq ele mesmo usa essa porta.

In [23]:
!./ngrok authtoken 2Lh7Zu0kns54bTSQCTrDDYkyvNH_71s7E3phX7wCyU4nuAFpH
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
https://f206-34-86-124-12.ngrok.io


Para conseguir pegar os dados direto do drive pq a base de dados é relativamente grande, tem cerca de 25 M, então demora muito pra subir pro colab. Então dessa maneira podemos autenticar no drive, basta conectar a conta do drive, permitir que ele acesse. 

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1ª passo é ler os dados que precisamos aplicar ao nosso modelo

In [29]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, TimestampType 

schema_remetente_destinatario = StructType([
    StructField('nome', StringType()),
    StructField('banco', StringType()),
    StructField('tipo', StringType())
])

schema_base_pix = StructType([
    StructField('id_transacao', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('remetente ', schema_remetente_destinatario),
    StructField('destinatario', schema_remetente_destinatario),
    StructField('transaction_date', StringType()),
    StructField('chave_pix', StringType()),
    StructField('categoria', StringType()),
    StructField('fraude', IntegerType())
])

caminho_json = 'drive/MyDrive/Colab Notebooks/case_final.json'

df = spark.read.json(
    caminho_json,
    schema = schema_base_pix,
    timestampFormat = "yyyy-MM-dd HH:mm:ss"
)

Py4JJavaError: ignored

Podemos importar todos os tipos que o spark tem, sem precisar referenciar um a um:

from pyspark.sql.types import *

2º Passo: ver o que tem dentro do arquivo

In [ ]:
df.printSchema()

df.show()

Precisamos verificar quais colunas preciso pra fazer o modelo, descarto as que não fazem diferença no modelo

Vamos excluir as colunas que não servem pro modelo e  transformar o que era estrutura em novas colunas, que é justamente fazer o flatten. Pra isso preciso criar um dicionário aonde o primeiro parâmetro é o nome da nova coluna e o 2º parametro é o valor dessa coluna

Precisamos importar a função de col

A função drop é pra excluir a coluna 

In [ ]:
from pyspark.sql.functions import *

df_flatten = df.withColumns({
    'destinatario_nome': col('destinatario').getField('nome'),
    'destinatario_banco': col('destinatario').getField('banco'),
    'destinatario_tipo': col('destinatario').getField('tipo')
}).drop('remetente', 'destinatario')

O .getField vai dentro de uma estrutura e pega o valor referente ao valor passado. Ou seja, vai no dicionário e pega o valor dentro da chave

Verificando como ficou o schema dessa estrutura

In [ ]:
df_flatten.printSchema()

Como a coluna de destinatário foi duplicado, podemos também dropar a coluna de destinatário como mostrado acima

Agora vamos fazer a modelagem aplicando algoritmo de machine learn. 

Na coluna chave_pix temos valores categoricos, o ideal é que a coluna fique numerica para depois poder aplicar regressão logistica.

Usando uma biblioteca do pyspark pra pegar um conjunto de colunas de entradas, tenho que referenciar qual as saidas dessas colunas e ele transforma esssas colunas categoricas em valores numericos

In [31]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(
    inputCols=[
        'destinatario_nome',
        'destinatario_banco',
        'destinatario_tipo',
        'categoria',
        'chave_pix'
     ],
     outputCols=[
        'destinatario_nome_index',
        'destinatario_banco_index',
        'destinatario_tipo_index',
        'categoria_index',
     _  'chave_pix_index'
     ]
)

SyntaxError: ignored

Agora preciso transformar os dados em dataframe. Usar a função fit que vai receber como parametro o dataframe que vai ser trasformado.

In [ ]:
df_index = indexer.fit(df_flatton).transform(df_flatton)

Para ver se funcionou, vemos qual schema do data frame novo

In [ ]:
df_index.printSchema()

df_index.show()

Observamos que foram criadas novas colunas com valores numericos e cada coluna se referencia a um valor específico, por exemplo: o destinatario_banco de valor 4 tem valor valor caixa, nubank tem valor 2. Simplesmente ele criou um mapeamento entre os bancos pra um valor numerico, isso pra poder treinar o nosso modelo.

O próximo passo pra executar o algotimo de machine learn é criar dois dataframes. Um que seja so com fraudes e outro sem fraudes. Para isso a gente pode usar a função de filtro, pegando dataframe atual e filtrar os valores que forem 1, que é fraude.

In [ ]:
is_fraud = df_index.filter('fraude == 1')  #filter(col('fraude') == 1)

no_fraud = df_index.filter('fraude == 0')  

Precisa pegar uma amostra aleatória de dentro do dataframe dos dados que não é fraude usando sample, vou pegar 1% das minhas transações e seed que é um numero aleatório, por exemplo 123 pra ele gerar uma base aleatória.

In [32]:
no_fraud = no_fraud.sample(False, 0.01, seed=123)

NameError: ignored

Proxima coisa é unir o dataframe antigo de fraude com a amostra que eu fiz de não fraude.Para isso vamos usar o concat

In [ ]:
df_concat = no_fraud.union(is_fraud)

df = df_concat.sort('transaction_date')

df.count()

Proximo passo é separa uma base de treino e teste 

Criando nossa base de treino e teste. O random Split ele divide o dataframe em dois dataframes separados e eu posso mandar pra ele uma lista, pro primeiro 70% vai pra treino e 30% test, e o seed um numero aleatorio

In [ ]:
train, test = df.randomSplit([0.7,0.3], seed= 123)

Agora podemos ver quantos dados foram pra cada um pra ver se 70% dos dados foi pra treino e 30% foi pra teste

In [ ]:
print('train', train.count(), 'test', test.count())

Agora precisamos que na nossa base de treino não tenha erros. Logo precisamos criar uma nova coluna pra provar pro dataframe que essa linha é uma fraude essa linha não é. Pra isso vamos criar uma udf, que é basicamente digamos que vc queria executar um código python mas dentro do spark paralelizando os dados. Lembrando que sempre que se executa um código em python ele não paraleliza os dados. Somente quando se faz isso através do dataframe. 

O que a udf faz é identificar se a linha é uma fraude ou não através de um lambda, que é uma função específica do python que intera sobre o valor que vai entrar e faz uma ação em uma linha. Ele pega o valor que vai entrar e dependendo da condição vai retornar um valor ou outro.

In [ ]:
is_fraud = udf(lambda fraud: 1.0 if fraud > 0.0, DoubleType())

train = train.withColumn('is_fraud', is_fraud(train.fraude))

train = train.withColumn('is_fraud', is_fraud(train.fraude)) foi pra criar uma nova coluna que de fato vai ser usada no modelo

In [ ]:
train.printSchema()

Agora vamos importar as ferramentas de pyspark, isso tem na documentação do sparkML

Como regressão logistica é um modelo de classificação, iremos importar lá da biblioteca de classification. Agora precisamos importar o VectorAssembler e por ultimo o Pipeline

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline


Agora vamos criar cada um, primeira mente vamos criar um vectorAssembler, vai pegar todos os valores de uma linha vai unir em uma coluna só todos aqueles valores de uma lista.

Então precisams criar uma variavel Assembler e vamos passsar dois parametros - inputCols, que é onde vmos pegar todas as nossas colunas de treinamento que queira unir 

In [ ]:
assembler = VectorAssembler(
    inputCols = [x for x in train.columns if x not in ('transaction_date', 'fraud', 'is_fraud')],
    outputCol = 'feature'
)

inputCols = [x for x in train.columns if x not in ('')] vai passar de coluna a coluna e vai colocar nessa lista de inputCols, porém somente as colunas que não foram a transaction_date pq não quero que elas se tornem vetor

Agora vai pegar todas as colunas que não são as 3 de cima e vou criar outra coluna que vai se chamar feature. Ou seja, vai pegar todas as coluna e transformar em um vetor de valores.

Foi criado os destinatários index anteriormente, porém temos os dados duplicados, então precisar dropar eles

In [ ]:
assembler = VectorAssembler(
    inputCols = [x for x in train.columns if x not in ('transaction_date', 'fraud', 'is_fraud', 'destinatario_nome',
                                                       'destinatario_banco', 'destinatario_tipo', 'chave_pix', 'categoria')],
    outputCol = 'features'
)

outputCol = 'features' deve ser escrito assim e não feature, atenção!

Agora precisamos criar o modelo de regrassão logistica

Vamos referenciala com lr e vamos instaciar o objeto que importamos

setParams vai setar os paramentros da nossa regressão logistica

O maxIter é o maximo de interações que podemos fazer, é facultativo

labelCol que é o que vai definir se a nossa transação é fraudulosa ou não

predictionCol é a coluna que vai criar com resultado da predição que ela fez


In [ ]:
lr = LogisticRegression().setParams(
    maxIter = 100000,
    labelCol = 'is_fraud',
    predictionCol='prediction'
)

Note que essa funções ainda não foram feitos no dataframe, pra isso precisa primeiro executar o assembler de depois o lostic Regression pra isso vamos usar i pip line que usamos lá em cima e ele leva apenas um parametro que é o stage que é uma lista que vai colocar a ordem do que vai ser executado...por ultimo fazer um fit pra treinar o modelo, vai pegar o dataset de input e vai colocar dentro do modelo

In [ ]:
model = PipeLine(stages=[assembler, lr]).fit(train)

Agora vamos testar pra ver se a previsão foi correta

Podemos criar um novo dataframe, e do nosso modelo vamos usar a função transform, ou seja, vamos pegar um dataframe de entrada e verificar se existe uma fraude ou não. O primeiro dataframe vai ser o de teste que criamos lá atrás, que é um teste aleatorio

In [ ]:
predicted = model.transform(test)

In [ ]:
predicted.show()

O retorno desse dataframe vai ser o prorpio dataframe, porém com uma nova coluna que foi criada com nosso modelo. Nesse caso os primeiros casos foram identifcados como fraude. Agora como vamos verificar a quantidade de fraudes que foi acertada ou não.

Vemos a matriz de acerto desse modelo, vamos pegar esse dataframe e vamos modificar uma coluna e aplicar o lambda que foi criado anteriormente

In [ ]:
predicted = predicted.withColumn('is_fraud', is_fraud(predicted.fraud))

Estamos pegando e criando uma nova coluna is_fraud que ela não existe no df, e estamos pegando o valor que setamos la no dataset que identificamos anteriormente. Esse valor predicted.fraud é o correto, agora vamos fazer um crosstab entre essa coluna predicted.fraud que é a correta e a predição pra ver se funcionou

In [ ]:
predicted.crosstab('is_fraud', 'prediction').show()

criada a nossa matriz 

Agora vamos criar um exemplo nosso pra vermos como que vai a predição do algoritmo. Precisamos criar uma coluna com nome e valor